In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import convolve
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone

# Load Data
from tensorflow.examples.tutorials.mnist import input_data
# https://github.com/zalandoresearch/fashion-mnist#get-the-data
fashion_data = input_data.read_data_sets('data/fashion')



X = fashion_data.train.next_batch(10000)[:1]
Y = fashion_data.train.next_batch(10000)[1:]

X = np.asarray(X, 'float32')
Y = np.asarray(Y, 'float32').T
X = np.squeeze(X)

X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

In [ ]:
f, axes = pyplot.subplots(2, 2)
axes[0, 0].imshow(X_train[:1].reshape((28, 28)))
pyplot.gcf().set_size_inches(10, 10)  

In [ ]:
# Models we will use
logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1,
                                           multi_class='multinomial')
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

# #############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.1
rbm.n_iter = 20
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000

# Training RBM-Logistic Pipeline
rbm_features_classifier.fit(X_train, Y_train)

# Training the Logistic regression classifier directly on the pixel
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

# #############################################################################
# Evaluation

Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))


In [ ]:
LABEL_NAMES = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boots']


from matplotlib import pyplot
%matplotlib inline

def plot_predictions(images, predictions):
    n = images.shape[0]
    nc = int(np.ceil(n / 4))
    f, axes = pyplot.subplots(nc, 4)
    for i in range(nc * 4):
        y = i // 4
        x = i % 4
        axes[x, y].axis('off')

        label = LABEL_NAMES[np.argmax(predictions[i])]
        confidence = np.max(predictions[i])
        if i > n:
            continue
        axes[x, y].imshow(images[i].reshape((28, 28)))
        axes[x, y].text(0.5, 0.5, label + ' %.3f\n' % confidence, fontsize=10)

    pyplot.gcf().set_size_inches(10, 10)  



In [ ]:
plot_predictions(np.squeeze(X[:16]), 
                 rbm_features_classifier.predict(X_test[:16]))